# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('../WeatherPy/output_data/weather_data.csv')
weather_df.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature °F,Humidity (%),Cloud Coverage (%),Wind Speed (MPH)
0,0,Puerto Ayora,-22.313712,-105.200071,76.37,94.0,75.0,1.14
1,1,Ushuaia,-79.249209,-19.108186,77.00,27.0,0.0,20.71
2,2,Mataura,-51.884326,-154.946626,50.00,89.0,100.0,13.60
3,3,Lagoa,51.572116,-29.995326,58.15,64.0,86.0,36.26
4,4,East London,-56.638447,43.250809,76.17,87.0,8.0,11.88


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=gkey)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(weather_df[['Latitude', 'Longitude']], weights= weather_df['Humidity (%)'], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_df = weather_df.loc[(weather_df['Temperature \N{DEGREE SIGN}F'] > 65) & 
                          (weather_df['Temperature \N{DEGREE SIGN}F'] < 90) &
                          (weather_df['Humidity (%)'] < 50) &
                          (weather_df['Cloud Coverage (%)'] == 0)
                         ]
print(ideal_df.shape)
ideal_df

(12, 8)


,Unnamed: 0,City,Latitude,Longitude,Temperature °F,Humidity (%),Cloud Coverage (%),Wind Speed (MPH)
1,1,Ushuaia,-79.249209,-19.108186,77.00,27.0,0.0,20.71
159,159,Rio Gallegos,-51.194162,-69.472729,78.80,29.0,0.0,10.36
258,258,Tessalit,20.216496,0.415949,71.64,17.0,0.0,9.53
280,280,Salalah,13.560042,60.943091,77.00,27.0,0.0,6.91
293,293,Bilma,17.948643,14.973006,70.47,19.0,0.0,6.38
397,397,Tomatlan,18.569173,-106.991266,82.92,45.0,0.0,9.44
493,493,Dori,14.198919,0.159684,80.11,13.0,0.0,9.89
497,497,Taoudenni,20.894031,-3.111905,69.96,19.0,0.0,11.10
511,511,Miguel Auza,24.515852,-103.701885,74.71,16.0,0.0,13.02
518,518,Christchurch,-48.652984,177.876053,65.88,32.0,0.0,12.66


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
hotel_df = ideal_df
hotel_df['Hotel Name'] = ""
hotel_df = hotel_df.reset_index()

C:\Users\Owner\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [32]:
# # geocoordinates
# target_coordinates = "44.9778, 93.2650"
# target_radius = 500000000
# target_type = "lodging"


# # set up a parameters dictionary
# params = {
#     "location": target_coordinates,
#     "radius": target_radius,
#     "type": target_type,
#     "key": gkey
# }

# # base url
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# # run a request using our params dictionary
# response = requests.get(base_url, params=params).json()
# response


# geocoordinates
target_coordinates = ""
target_radius = 10000000
target_search = "hotel",
target_type = "lodging"

for index, row in hotel_df.iterrows():
    
    target_coordinates = f'{row["Latitude"]},{row["Longitude"]}'

    # set up a parameters dictionary
    params = {
    "location": target_coordinates,
    "rankby": "distance",    
    "type": target_type,   
    "radius": target_radius,
    "keyword": target_search,
    "key": gkey
        
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
#     print(target_coordinates)
    if response['status'] == 'ZERO_RESULTS':
        print(f'#{index} There are no results for {row["City"]}')
    else:
        print(f'#{index} There are results for {row["City"]}')

#0 There are results for Ushuaia
#1 There are results for Rio Gallegos
#2 There are results for Tessalit
#3 There are results for Salalah
#4 There are results for Bilma
#5 There are results for Tomatlan
#6 There are results for Dori
#7 There are results for Taoudenni
#8 There are results for Miguel Auza
#9 There are results for Christchurch
#10 There are results for Biltine
#11 There are results for Suleja


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
